In [ ]:
import numpy as np
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from sklearn.datasets import fetch_20newsgroups
import pandas as pd

import matplotlib.pyplot as plt
plt.switch_backend('agg')
%matplotlib inline

In [ ]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [ ]:
def load_embeddings(filename):
    embeddings_index = {}
    with open(filename,encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    print('Total %s word vectors in %s' % (len(embeddings_index), filename))
    
    return embeddings_index

In [ ]:
def create_embedding_layer(word_index, embeddings_index):
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    
    return Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,trainable=True)

In [ ]:
def create_model(embedding_layer, macronum):
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    l_cov1= Conv1D(128, 5, activation='relu')(embedded_sequences)
    l_pool1 = MaxPooling1D(5)(l_cov1)
    l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
    l_pool2 = MaxPooling1D(5)(l_cov2)
    l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
    l_pool3 = MaxPooling1D(35)(l_cov3)  # global max pooling
    l_flat = Flatten()(l_pool3)
    l_dense = Dense(128, activation='relu')(l_flat)
    preds = Dense(len(macronum), activation='softmax')(l_dense)
    
    model = Model(sequence_input, preds)
    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

    print("Simplified convolutional neural network")
    model.summary()
    cp = ModelCheckpoint('model_cnn.hdf5',monitor='val_acc',verbose=1,save_best_only=True)
    return (model, cp)

In [ ]:
def load_twenty_newsgroup():
    newsgroups_train = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

    df = pd.DataFrame([newsgroups_train.data, newsgroups_train.target.tolist()]).T
    df.columns = ['text', 'target']
    
    return df

In [ ]:
def preprocess_text(text):
    return text

In [ ]:
def load_data():
    texts = []
    labels = []

    df = load_twenty_newsgroup()
    
    texts = df['text'].apply(preprocess_text)
    labels = df['target']
    
    macronum=sorted(set(df['target']))
    
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)

    word_index = tokenizer.word_index
    print('Number of Unique Tokens',len(word_index))

    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    labels = to_categorical(np.asarray(labels))
    print('Shape of Data Tensor:', data.shape)
    print('Shape of Label Tensor:', labels.shape)

    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    data = data[indices]
    labels = labels[indices]
    nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

    x_train = data[:-nb_validation_samples]
    y_train = labels[:-nb_validation_samples]
    x_val = data[-nb_validation_samples:]
    y_val = labels[-nb_validation_samples:]
    
    return (x_train, y_train, x_val, y_val, word_index, macronum)

In [ ]:
embeddings_filename = 'glove.6B.' + str(EMBEDDING_DIM) + 'd.txt'
embeddings_index = load_embeddings(embeddings_filename)

In [ ]:
x_train, y_train, x_val, y_val, word_index, macronum = load_data()

In [ ]:
embedding_layer = create_embedding_layer(word_index, embeddings_index)
model, cp = create_model(embedding_layer, macronum)

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=10, batch_size=2, callbacks=[cp])

In [ ]:
fig2=plt.figure()
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves : CNN',fontsize=16)
fig2.savefig('accuracy_cnn.png')
plt.show()